In [34]:
import pandas as pd
import os
import datetime as dt
import re
import xlsxwriter
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px

In [35]:
# Загрузка ежедневного отчета
daily = pd.read_excel('Обратная связь.xlsx', 
                      sheet_name="Основной лист", 
                      parse_dates=False, 
                      engine='openpyxl',
                      keep_default_na=True, 
                      convert_float=True, 
                      dtype={"Ресторан № (собеседование)": str, "Ресторан № (работа)": str}, 
                      na_values=[pd.NaT])

daily['Ресторан № (собеседование)'] = daily['Ресторан № (собеседование)'].apply(add_zero)
daily['Ресторан № (работа)'] = daily['Ресторан № (работа)'].apply(add_zero)

In [36]:
# Загрузка информации распределения дивизионов и регионов
divizions = pd.read_excel('распределение ресоранов по девизионам.xlsx', sheet_name="Лист1", parse_dates=True, engine='openpyxl', 
                               keep_default_na=True, convert_float=True, dtype={'код ресторана': str},
                              na_values=[pd.NaT])
divizions =  divizions.drop('город', axis=1)

In [37]:
# Загрузка заявок на подбор (с проставленным номером ресторана и приоритетностью заявки)
zayavki = pd.read_excel('Заявки ЧБР НУ Курьеры 28.12.2020.xlsx', 
                        sheet_name="28.-31.12.20", 
                        parse_dates=False, 
                        engine='openpyxl',
                        keep_default_na=True, 
                        convert_float=True, 
                        dtype={"рест": str})

zayavki = zayavki.dropna(thresh=1)
zayavki = zayavki.fillna(value={'Приоритет': 0})
zayavki = zayavki[['рест', 'Приоритет', 'Количество (число)']] \
            .rename(columns={'Количество (число)': 'Количество в заявке'})

In [38]:
# Смерживаем полученные таблицы
merged_table = pd.merge(divizions, daily, left_on='код ресторана', right_on='Ресторан № (работа)') \
                    .rename(columns={'Дивизион_x': 'Дивизион'})
merged_table = pd.merge(merged_table, zayavki, left_on='код ресторана', right_on='рест') 

In [39]:
merged_table.columns

Index(['код ресторана', 'Дивизион', 'регион', 'Рекрутер', 'ФИО',
       'Телефон(в формате ОРИС)', 'Возраст', 'Гражданство',
       'Телефонное интервью (число)', 'Источник', 'Дата собеседования',
       'Ресторан № (работа)', 'Ресторан № (собеседование)', 'ТУ', 'Регион',
       'Дивизион_y', 'Должность', 'Дошел/Не дошел',
       'Обратная связь от ресторана', 'Обратная связь от кандидата',
       'Направлен на оформление', 'Оформился ДА/НЕТ', 'Прошел МО ДА/НЕТ',
       'Вышел в ресторан', 'Смена 4 часа ДА/НЕТ', 'Дата первой смены',
       'Unnamed: 23', 'рест', 'Приоритет', 'Количество в заявке'],
      dtype='object')

In [50]:
# Группируем полученную таблицу
all_groupped = merged_table.groupby(['Телефонное интервью (число)', 'Дивизион', 'регион'], as_index=False) \
                        .aggregate({'ФИО': 'count', 'Приоритет': 'sum', 'Количество в заявке':'sum'}) \
                        .rename(columns={'ФИО': 'направлено от ОП'}) 

In [51]:
all_groupped.head()

,Телефонное интервью (число),Дивизион,регион,направлено от ОП,Приоритет,Количество в заявке
0,2020-12-17,Евразия,NWEST SOUTH,1,0.0,2.0
1,2020-12-17,Евразия,SYBER,2,0.0,9.0
2,2020-12-17,Евразия,URALS NORTH,10,0.0,43.0
3,2020-12-17,Евразия,URALS SOUTH,1,1.0,3.0
4,2020-12-17,ММО,MSK MO NO-EAST,13,0.0,38.0


In [52]:
# Визуализация данных за все время

df = px.data.gapminder()

fig = px.scatter(all_groupped, 
                 x='Телефонное интервью (число)', 
                 y="Количество в заявке", 
                 size="направлено от ОП", 
                 color="Приоритет",
                 hover_name="регион", 
                 title=f"Направленные от ОП по регионам")
fig.show()

In [53]:
# Выбираем нужный день
day = '2021-01-14'
for_day = all_groupped[all_groupped['Телефонное интервью (число)'] == day]

In [54]:
# Визуализация данных за день
df = px.data.gapminder()

fig = px.scatter(for_day, x="регион", y="Количество в заявке", size="направлено от ОП", color="Приоритет",
                 hover_name='Дивизион', title=f"Направленные от ОП по регионам {day}")
fig.show()